# Finetuning using Axolotl

This notebook is an minimal example of how to finetune a LLM using [Axolotl](https://github.com/axolotl-ai-cloud/axolotl). Axolotl is a CLI tool that uses config files for different methods of LLM finetuning. We created a Python wrapper around the CLI for the end-to-end workflow for this process.

In the example below, we show how you can define or load finetuning configurations to replicate the fine-tuning process from our paper, start a fine-tuning job, and push the model to Hugging Face.s

## Setup

Make sure to run this notebook in a system with enough compute resources to run the finetuning, and follow the setup instructions in the README to install axolotl and related libraries.

Let's start with loading the code components we need. The `FinetuneConfig` class holds configurations, and the `Finetune` class is used to create and run a finetuning job.

In [1]:
import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from finetune import Finetune, FinetuneConfig

[2025-01-21 17:35:35,201] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Also make sure to login to Hugging Face Hub to save the output model.

In [2]:
from huggingface_hub import login
login()

## Configuration
We then load up a config to perform QLoRA finetuning of `meta-llama/Llama-2-7b-chat-hf` from in a config file stored locally. Optionally, we can assign a new field `hub_model_id`, indicating the Hugging Face model the finetuned LLM will be pushed to.

For experiments in the paper we did:
1. LoRA finetuning. The config file for LoRA finetune is `altus/finetune/examples/llama-2/lora.yml`.
2. SFT. The config file is `altus/finetune/examples/llama-2/fft_optimized.yml`


Next, we load a configuration file to perform LoRA fine-tuning of `meta-llama/Llama-2-7b-chat-hf`. The configuration file is stored locally. Optionally, a new field `hub_model_id` can be assigned to specify the Hugging Face model where the fine-tuned LLM will be pushed.

For the experiments presented in the paper, we performed the following:

1. **LoRA** Fine-tuning: The configuration file for LoRA fine-tuning is located at:

`altus/finetune/examples/llama-2/lora.yml`.

2. **SFT** (Supervised Fine-tuning): The configuration file for SFT is located at:

`altus/finetune/examples/llama-2/fft_optimized.yml`

In [3]:
import yaml

# Specify the path to your YAML file
file_path = os.path.join(os.getcwd(), '..', 'finetune/examples/llama-2/lora.yml')

# Open the file and load the data
with open(file_path, encoding='utf-8') as file:
    config_dict = yaml.safe_load(file)  # Load the existing data

config_dict['base_model'] = 'meta-llama/Llama-2-7b-chat-hf'
config_dict['hub_model_id'] = 'vijil/my_lora_tune'  # Add or update the model_id to push the trained model
config_dict['eval_sample_packing'] = False

In [4]:
config_dict

{'base_model': 'meta-llama/Llama-2-7b-chat-hf',
 'model_type': 'LlamaForCausalLM',
 'tokenizer_type': 'LlamaTokenizer',
 'load_in_8bit': True,
 'load_in_4bit': False,
 'strict': False,
 'datasets': [{'path': 'mhenrichsen/alpaca_2k_test', 'type': 'alpaca'}],
 'dataset_prepared_path': None,
 'val_set_size': 0.05,
 'output_dir': './lora-out',
 'sequence_len': 4096,
 'sample_packing': True,
 'pad_to_sequence_len': True,
 'adapter': 'lora',
 'lora_model_dir': None,
 'lora_r': 32,
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_target_linear': True,
 'lora_fan_in_fan_out': None,
 'wandb_project': None,
 'wandb_entity': None,
 'wandb_watch': None,
 'wandb_name': None,
 'wandb_log_model': None,
 'gradient_accumulation_steps': 4,
 'micro_batch_size': 2,
 'num_epochs': 4,
 'optimizer': 'adamw_bnb_8bit',
 'lr_scheduler': 'cosine',
 'learning_rate': 0.0002,
 'train_on_inputs': False,
 'group_by_length': False,
 'bf16': 'auto',
 'fp16': None,
 'tf32': False,
 'gradient_checkpointing': True,
 'earl

Let's now load the config dict in the `FinetuneConfig` object.

In [5]:
# see all config options in './finetune/axolotl/examples/config.qmd'
config = FinetuneConfig(config_dict)

[2025-01-21 17:35:54,661] [DEBUG] [axolotl.normalize_config:79] [PID:3716] [RANK:0] bf16 support detected, enabling for this configuration.


/workspace/bash/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

[2025-01-21 17:35:54,938] [INFO] [axolotl.normalize_config:182] [PID:3716] [RANK:0] GPU memory usage baseline: 0.000GB (+0.682GB misc)


## Run the finetuning job
Now simply load up the config into a `FineTune` object and kick off the job.

In [6]:
# create a finetune object with the config and run
finetune = Finetune(config)

In [7]:
finetune.run() # start train

[INFO] Job ID: 2cb97706-d81e-11ef-9844-0242ac120002


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

[2025-01-21 18:12:02,650] [INFO] [accelerate.accelerator.log:61] [PID:3716] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2025-01-21 18:12:02,689] [INFO] [axolotl.utils.samplers.multipack._len_est:184] [PID:3716] [RANK:0] packing_efficiency_estimate: 0.98 total_num_tokens per device: 414041
[2025-01-21 18:14:10,695] [INFO] [accelerate.accelerator.log:61] [PID:3716] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2025-01-21 18:16:18,747] [INFO] [accelerate.accelerator.log:61] [PID:3716] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.
[2025-01-21 18:18:26,798] [INFO] [accelerate.accelerator.log:61] [PID:3716] The used dataset had no length, returning gathered tensors. You should drop the remainder yourself.


/workspace/bash/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2025-01-21 18:18:41,447] [INFO] [axolotl.train.log:61] [PID:3716] [RANK:0] Training Completed!!! Saving pre-trained model to ./lora-out


/workspace/bash/envs/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/320M [00:00<?, ?B/s]